In [ ]:
import radar
#%matplotlib tk
#%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import importlib
import cfar
import tracking

In [ ]:
importlib.reload(radar)
importlib.reload(cfar)

In [ ]:
from pathlib import Path
carpetaFiguras = Path("figuras/")
carpetaFiguras.mkdir(parents = True,exist_ok = True)

In [ ]:
#cod1=radar.Codigo_pseudoaleatorio(13,30,1)
cod1=radar.Codigo_pseudoaleatorio(semilla = 202305041529+4,numero_bits=22,numero_muestras_bit=2)

In [ ]:
def cod_y_correlacion():
    plt.figure(1,figsize=(10,6),dpi=100,layout='tight')
    plt.clf()
    plt.subplot(2,1,1)
    plt.title("Pulso codificado (secuencia pseudoaleatoria)")
    ycod=cod1.muestras_codigo
    xs=np.arange(-len(ycod)+1,len(ycod))
    ycorr=np.correlate(ycod,ycod,"full")
    plt.stem(xs,np.concatenate((np.zeros_like(ycod[:-1]),ycod)))
    plt.subplot(2,1,2)
    plt.title("Autocorrelación de Pulso codificado")
    plt.stem(xs,ycorr)
    plt.plot(xs,ycorr,'b--')
    plt.xlabel("lag")
    plt.savefig("figuras/autocorrelacion-pulso-codificado.png",dpi=600)
    plt.show()
cod_y_correlacion()

In [ ]:
rad1 = radar.Canal_radar(frec_muestreo=200000,frec_portadora=7e6,pulso=cod1.muestras_codigo,guarda=15000,rango_maximo=400e3, semilla_rng=202305041529+4)
comp1 = radar.Compresor_pulso(rad1)
simulador = radar.Simulador_escenario(rad1)
escenario = [radar.mru(140000,-4), radar.mru(75000,12), radar.mru(100000,-6), radar.mru(55000,0)]
exploraciones = simulador.exploracion(escenario,0,60,120, snr_rango_max=1/3)

In [ ]:
comp1.amplitud_cuadrada_correlacion(exploraciones[5]).shape

In [ ]:
#plot de amplitud cuadrada de la correlación para algunos tiempos en función del rango
def fig14():
  plt.figure(figsize=(12,20),dpi=100)
  plt.title("Amplitud cuadrada de la correlación a diferentes tiempos")
  #plt.xlabel("Rango (en km)")
  #plt.ylabel("Amplitud")
  for i in range(10):
    plt.subplot(10,1,i+1)
    plt.xlabel("Rango (en km)")
    plt.ylabel("Amplitud")
    plt.plot(comp1.amplitud_cuadrada_correlacion(exploraciones[i*10]))
    plt.savefig("figuras/amplitud-correlacion.png",dpi=600)
  plt.show()
fig14()

In [ ]:
def fig15(compresor_prueba,exploraciones):
  correlacion = compresor_prueba.amplitud_cuadrada_correlacion(exploraciones[30])
  radar ={"frec_muestreo":compresor_prueba.frec_muestreo, # en 1/s
          "ancho_pulso":compresor_prueba.pulso.size, # adimensional
          "distancia_ciega": compresor_prueba.rango_minimo_km, # km
          "rango_por_unidad_de_retardo":1.5e5} # km/s
  detector={"ancho_ventana":30,
            "k":(30*6)//7}
  detector["T"]=cfar.calcula_t_para_pfa(detector["ancho_ventana"],detector["k"])
  rangos,detalle= cfar.detectarOsCfar(radar,correlacion,detector)
  plt.figure(figsize=(16,6),dpi=150)
  plt.subplot(2,1,1)
  plt.plot(correlacion,'.--')
  detecciones = detalle["posiciones"]
  plt.plot(detecciones,correlacion[detecciones],'r+')
  plt.plot(detalle["centros"],detalle["umbrales"],'.--')
  plt.subplot(2,1,2)
  plt.plot(rangos,np.ones_like(rangos),'ro')
  plt.savefig("figuras/cfar-detecciones.png",dpi=600)
  #plt.xlim(0,100)
  plt.show()
fig15(comp1,exploraciones)

In [ ]:
plt.subplot(2,1,1)
plt.plot(np.real(exploraciones[100]))
plt.subplot(2,1,2)
plt.plot(np.imag(exploraciones[100]))

In [ ]:
sim = simulador.simula(escenario,0,.5,120,120)

In [ ]:
def fig16(exploracion,comp_pulso):
    plt.figure(16,figsize=(10,10))
    corr =  np.array([comp_pulso.amplitud_cuadrada_correlacion(exploracion[i,:]) for i in range(exploracion.shape[0])])
    corr = np.fft.fftshift(corr,0)
    R,F = np.meshgrid(np.arange(corr.shape[1]),np.fft.fftshift(np.fft.fftfreq(corr.shape[0],0.5)))
    ax=plt.axes(projection='3d')
    ax.plot_surface(R,F,corr)
    plt.show()
fig16(sim[100],comp1)

In [ ]:
# kf = tracking.KF(initial_x=0.0, initial_v=1.0, accel_variance=0.1)
# kf = KF(initial_x=0.0, initial_v=1.0, accel_variance=0.1)

def fn1(compresor,exploraciones,dt=0.1,varDet=.1,varAcel=.1,tol=1,tolPred=10):
    radar ={"frec_muestreo":compresor.frec_muestreo, # en 1/s
        "ancho_pulso":compresor.pulso.size, # adimensional
        "distancia_ciega": compresor.rango_minimo_km, # km
        "rango_por_unidad_de_retardo":1.5e5} # km/s
    detector={"ancho_ventana":30,
                "k":(30*6)//7}
    detector["T"]=cfar.calcula_t_para_pfa(detector["ancho_ventana"],detector["k"])
    seguimiento = tracking.GestorDeTrayectos()
    for i in range(exploraciones.shape[0]):
        a2 = compresor.amplitud_cuadrada_correlacion(exploraciones[i,:])
        rangos,_ = cfar.detectarOsCfar(radar,a2,detector)
        seguimiento.actualizaTrayectos(rangos,dt,varDet,varAcel,tol,tolPred)
    return seguimiento.copiaTrayectos()


trayectos=fn1(comp1,exploraciones,varDet=1,varAcel=1,tol=3)


In [ ]:
help (sorted)

In [ ]:
plt.figure(1)
for i,trayecto in enumerate(sorted(trayectos,key=lambda x:len(x[:,0]))):
    datos_trayecto = np.asanyarray(trayecto)
    t = datos_trayecto[:,0]
    x = datos_trayecto[:,1]
    var_x= datos_trayecto[:,2]
    v= datos_trayecto[:,3]
    var_v= datos_trayecto[:,4]
    if len(t) > 73:
        plt.plot(t,x,linewidth=2)
    else:
        plt.plot(t,x,linewidth=1,linestyle="--")
plt.ylim(0,200)
#plt.savefig("figuras/trayectos.png",dpi=600)
plt.show()
